In [2]:
import sys
sys.path.append('/Users/danielcopeland/Library/Mobile Documents/com~apple~CloudDocs/MIT Masters/DRL/LABx/RADARTreePose')

from data_scripts.data_type_classes.RADARDataCapture import FMCWRADARDataCapture
from data_scripts.data_type_classes.MOCAPDataCapture import MOCAPDataCapture
from data_scripts.data_type_classes.FPDataCapture import FPDataCapture
from data_scripts.data_type_classes.tsv_processor_knee_angles import TSVProcessorKnee
import h5py
import os
import numpy as np
import pandas as pd
import imageio
import matplotlib.pyplot as plt
from datetime import datetime
import statistics
from matplotlib import animation
import ast

In [2]:
# # Define the lists of parameters
# participants = ['01', '02', '03', '04', '05', '08', '10', '12', '13', '14', '15', '16', '18', '22', '24']
# # participants = ['12']
# movements = ["MNTRL", "MNTRR"]
# # movements = ["MNTRR"]
# versions = ["V1", "V2", "V3"]
# # versions = ["V3"]

time_labels_df_path = '/Users/danielcopeland/Library/Mobile Documents/com~apple~CloudDocs/MIT Masters/DRL/LABx/RADARTreePose/data/csvs/MOCAP_FP_RADAR_FU_Stable_Break_FD_TIME_FRAMES_v3.csv'
time_labels_df = pd.read_csv(time_labels_df_path)

print(time_labels_df.head())

    RADAR_capture  MOCAP_Start_Time  MOCAP_End_Time  RADAR_Start_Frame  \
0  01_MNTRL_RR_V1              7.33           30.57                194   
1  01_MNTRL_RR_V1              7.33           30.57                194   
2  01_MNTRL_RR_V1              7.33           30.57                194   
3  01_MNTRL_RR_V2              7.37           30.40                191   
4  01_MNTRL_RR_V2              7.37           30.40                191   

   RADAR_End_Frame  Seconds_per_Frame  tx  is_final_tx  t_foot_up  t_stable  \
0              831            0.03645   1        False      10.22     11.19   
1              831            0.03645   2        False      18.25     19.05   
2              831            0.03645   3         True      26.24     27.03   
3              822            0.03646   1        False      10.48     11.05   
4              822            0.03646   2        False      18.41     19.07   

   t_break  t_foot_down  frame_foot_up  frame_stable  frame_break  frame_end  
0

Generate AoA for each Stability Period

In [12]:
output_folder_path = "/Volumes/FourTBLaCie/Yoga_Study_RADAR_AoA_Stability"

problem_list = []

for index, row in time_labels_df.iterrows():
    
    print(row)
    
    # if row['RADAR_capture'] != '12_MNTRL_RR_V1':
    #     continue
    
    participant = row['RADAR_capture'][:2]
    
    radar_file_path = f"/Volumes/FourTBLaCie/Yoga_Study_RADAR_Data_Renamed/{participant}/{row['RADAR_capture']}.h5"
    
    try:
        # Initialize the FMCWRADARDataCapture object
        RADAR_object = FMCWRADARDataCapture(file_path=radar_file_path)
    except Exception as e:
        problem_list.append((row['RADAR_capture'], row['tx'], e))
        continue
    
    # Load and save the data
    dataCubes = RADAR_object.load_and_save()

    # Process the data with AoA processing
    processed_data = RADAR_object.angle_of_arrival_processing(dataCubes)
    
    print(processed_data.shape)
    
    sub_selected_processed_data = RADAR_object.sub_select_AoA_DATA(processed_data)[0,:,:,:]
    
    print(sub_selected_processed_data.shape)
    
     # Convert the list of processed frames into a 3D numpy array
    processed_frames_array = np.array(sub_selected_processed_data)
    
    print(processed_frames_array.shape)
    
    frame_stable = int(row['frame_stable'])
    if row['is_final_tx']:
        frame_end = int(row['frame_end'])
    else:
        frame_end = int(row['frame_break'])
    
    stable_processed_frames_array = processed_frames_array[frame_stable:frame_end,:,:]
    
    print(stable_processed_frames_array.shape)
    
    # Ensure the output folder exists
    specific_output_folder_path = os.path.join(output_folder_path, participant)
    if not os.path.exists(specific_output_folder_path):
        os.makedirs(specific_output_folder_path)
    
    # Define the output file name for the processed data of the current transition
    output_filename = os.path.join(
        specific_output_folder_path,
        f"{row['RADAR_capture']}_AoA_tx{row['tx']}.npy"
    )
    # Save the 3D array to a binary file in NumPy .npy format
    np.save(output_filename, stable_processed_frames_array)
    
    print(f"saved {output_filename}")
    print("test")
    

    

RADAR_capture        01_MNTRL_RR_V1
MOCAP_Start_Time               7.33
MOCAP_End_Time                30.57
RADAR_Start_Frame               194
RADAR_End_Frame                 831
Seconds_per_Frame           0.03645
tx                                1
is_final_tx                   False
t_foot_up                     10.22
t_stable                      11.19
t_break                       14.28
t_foot_down                   15.05
frame_foot_up                   273
frame_stable                    300
frame_break                   384.0
frame_end                       405
Name: 0, dtype: object
(4, 1000, 256, 128)
(1000, 23, 13)
(1000, 23, 13)
(84, 23, 13)
saved /Volumes/FourTBLaCie/Yoga_Study_RADAR_AoA_Stability/01/01_MNTRL_RR_V1_AoA_tx1.npy
test
RADAR_capture        01_MNTRL_RR_V1
MOCAP_Start_Time               7.33
MOCAP_End_Time                30.57
RADAR_Start_Frame               194
RADAR_End_Frame                 831
Seconds_per_Frame           0.03645
tx                           

KeyboardInterrupt: 

Generate Stability Phase RDMs

In [25]:
actuator_df = pd.read_csv('/Users/danielcopeland/Library/Mobile Documents/com~apple~CloudDocs/MIT Masters/DRL/LABx/RADARTreePose/data/csvs/MOCAP_FP_RADAR_FU_Stable_Break_FD_TIME_FRAMES_v3.csv')

output_folder_path = "/Volumes/FourTBLaCie/Yoga_Study_RADAR_4Ch_BtwnActuatorCapture"
output_folder_path_one_channel = "/Volumes/FourTBLaCie/Yoga_Study_RADAR_1Ch"

problem_list = []

for index, row in time_labels_df.iterrows():
    
    if row['RADAR_capture'] != '12_MNTRL_RR_V1':
        continue
    capture_info = actuator_df[actuator_df['RADAR_capture']==row['RADAR_capture']]    
    RADAR_start_frame = int(capture_info['RADAR_Start_Frame'].iloc[0])
    RADAR_end_frame = int(capture_info['RADAR_End_Frame'].iloc[0])
    
    participant = row['RADAR_capture'][:2]
    
    radar_file_path = f"/Volumes/FourTBLaCie/Yoga_Study_RADAR_Data_Renamed/{participant}/{row['RADAR_capture']}.h5"
    
    try:
        # Initialize the FMCWRADARDataCapture object
        RADAR_object = FMCWRADARDataCapture(file_path=radar_file_path)
    except Exception as e:
        problem_list.append((row['RADAR_capture'], row['tx'], e))
        continue
    
    # Load and save the data
    dataCubes = RADAR_object.load_and_save()

    # Process the data with Range-Doppler processing
    processed_data = RADAR_object.range_doppler_processing(dataCubes)
    
    print(processed_data.shape)
    
    sub_selected_processed_data = RADAR_object.sub_select_RADAR_DATA(processed_data)
    
    print(sub_selected_processed_data.shape)
    
     # Convert the list of processed frames into a 3D numpy array
    processed_frames_array = np.array(sub_selected_processed_data)
    
    print(processed_frames_array.shape)
    
    # Ensure the output folder exists
    specific_output_folder_path = os.path.join(output_folder_path, participant)
    if not os.path.exists(specific_output_folder_path):
        os.makedirs(specific_output_folder_path)
    
    # Define the output file name for the processed data of the current transition
    output_filename = os.path.join(
        specific_output_folder_path,
        f"{row['RADAR_capture']}.npy"
    )
    # Save the 3D array to a binary file in NumPy .npy format
    np.save(output_filename, processed_frames_array)
    
    print(f"{row['RADAR_capture']} processed and saved")
    
    for i, channel in enumerate(processed_frames_array):
        print(channel.shape)
            # Ensure the output folder exists
        specific_output_folder_path_one_channel = os.path.join(output_folder_path_one_channel, participant)
        if not os.path.exists(specific_output_folder_path_one_channel):
            os.makedirs(specific_output_folder_path_one_channel)
            
            # Define the output file name for the processed data of the current transition
        output_filename_channel = os.path.join(
            specific_output_folder_path_one_channel,
            f"{row['RADAR_capture']}_channel{i+1}.npy"
        )
        # Save the 3D array to a binary file in NumPy .npy format
        np.save(output_filename_channel, channel)
        print(output_filename_channel)
        
    

(4, 1000, 128, 128)
(4, 1000, 23, 13)
(4, 1000, 23, 13)
(1000, 23, 13)
/Volumes/FourTBLaCie/Yoga_Study_RADAR_1Ch/01/01_MNTRL_RR_V1_channel1.npy
(1000, 23, 13)
/Volumes/FourTBLaCie/Yoga_Study_RADAR_1Ch/01/01_MNTRL_RR_V1_channel2.npy
(1000, 23, 13)
/Volumes/FourTBLaCie/Yoga_Study_RADAR_1Ch/01/01_MNTRL_RR_V1_channel3.npy
(1000, 23, 13)
/Volumes/FourTBLaCie/Yoga_Study_RADAR_1Ch/01/01_MNTRL_RR_V1_channel4.npy
(4, 1000, 128, 128)
(4, 1000, 23, 13)
(4, 1000, 23, 13)
(1000, 23, 13)
/Volumes/FourTBLaCie/Yoga_Study_RADAR_1Ch/01/01_MNTRL_RR_V1_channel1.npy
(1000, 23, 13)
/Volumes/FourTBLaCie/Yoga_Study_RADAR_1Ch/01/01_MNTRL_RR_V1_channel2.npy
(1000, 23, 13)
/Volumes/FourTBLaCie/Yoga_Study_RADAR_1Ch/01/01_MNTRL_RR_V1_channel3.npy
(1000, 23, 13)
/Volumes/FourTBLaCie/Yoga_Study_RADAR_1Ch/01/01_MNTRL_RR_V1_channel4.npy
(4, 1000, 128, 128)
(4, 1000, 23, 13)
(4, 1000, 23, 13)
(1000, 23, 13)
/Volumes/FourTBLaCie/Yoga_Study_RADAR_1Ch/01/01_MNTRL_RR_V1_channel1.npy
(1000, 23, 13)
/Volumes/FourTBLaCie/Yoga

Generate Stability Phase AoA Heat Maps

Generate 4 Channel RDMs

In [28]:
output_folder_path = "/Volumes/FourTBLaCie/Yoga_Study_RADAR_4Ch_FUFD"

problem_list = []

for index, row in time_labels_df.iterrows():
    
    # if row['RADAR_capture'] != '12_MNTRL_RR_V1':
    #     continue
    
    participant = row['RADAR_capture'][:2]
    print(participant)
    print(row)
    
    radar_file_path = f"/Volumes/FourTBLaCie/Yoga_Study_RADAR_Data_Renamed/{participant}/{row['RADAR_capture']}.h5"
    
    try:
        # Initialize the FMCWRADARDataCapture object
        RADAR_object = FMCWRADARDataCapture(file_path=radar_file_path)
    except Exception as e:
        problem_list.append((row['RADAR_capture'], row['tx'], e))
        continue
    
    # Load and save the data
    dataCubes = RADAR_object.load_and_save()

    # Process the data with Range-Doppler processing
    processed_data = RADAR_object.range_doppler_processing(dataCubes)
    
    print(processed_data.shape)
    
    sub_selected_processed_data = RADAR_object.sub_select_RADAR_DATA(processed_data)
    
    print(sub_selected_processed_data.shape)
    
    file_name = row['RADAR_capture']+"_tx"+str(row['tx'])+"_"+str(row['is_final_tx'])
    
    print(file_name)
    
    start_end_frames = (row['Start_Frame'],row['End_Frame'])  # Assuming this is a string
    
    RADAR_object.process_and_save_FUFD_RADAR_data(data=sub_selected_processed_data, start_end_frames = start_end_frames, output_folder_path=output_folder_path, file_name=file_name)
    
    

01
Unnamed: 0                        0
RADAR_capture        01_MNTRL_RR_V1
tx                                1
is_final_tx                   False
Seconds_per_Frame          0.036454
Start_Time                    10.22
End_Time                      15.05
Start_Frame                     273
End_Frame                       405
t_stable                      11.19
t_break                       14.28
frame_stable                  300.0
frame_break                   384.0
Name: 0, dtype: object
(4, 1000, 128, 128)
(4, 1000, 23, 13)
01_MNTRL_RR_V1_tx1_False
(4, 132, 23, 13)
01_MNTRL_RR_V1_tx1_False processed and saved
01
Unnamed: 0                        1
RADAR_capture        01_MNTRL_RR_V1
tx                                2
is_final_tx                   False
Seconds_per_Frame          0.036454
Start_Time                    18.25
End_Time                      22.83
Start_Frame                     493
End_Frame                       619
t_stable                      19.05
t_break           

Generate 4 different captures of RADAR data per channel, and subsegment into Going Up (GOUP), Balanced (BLNC), Going Down (DOWN).

In [12]:
time_labels_df_path = '/Users/danielcopeland/Library/Mobile Documents/com~apple~CloudDocs/MIT Masters/DRL/LABx/RADARTreePose/data/csvs/MOCAP_FP_RADAR_FU_Stable_Break_FD_TIME_FRAMES.csv'
time_labels_df = pd.read_csv(time_labels_df_path)

print(time_labels_df.head())

   Unnamed: 0   RADAR_capture  tx  is_final_tx  Seconds_per_Frame  Start_Time  \
0           0  01_MNTRL_RR_V1   1        False           0.036454       10.22   
1           1  01_MNTRL_RR_V1   2        False           0.036454       18.25   
2           2  01_MNTRL_RR_V1   3         True           0.036454       26.24   
3           3  01_MNTRL_RR_V2   1        False           0.036464       10.48   
4           4  01_MNTRL_RR_V2   2        False           0.036464       18.41   

   End_Time  Start_Frame  End_Frame  t_stable  t_break  frame_stable  \
0     15.05          273        405     11.19    14.28           300   
1     22.83          493        619     19.05    22.18           515   
2     30.57          712        831     27.03      NaN           734   
3     15.05          276        401     11.05    14.27           292   
4     22.86          493        615     19.07    22.17           511   

   frame_break  
0        384.0  
1        601.0  
2          NaN  
3        380

In [14]:

for idx, row in time_labels_df.iterrows():
    radar_capture = row['RADAR_capture']
    print(radar_capture)
    participant = radar_capture[:2]
    tx_number = row['tx']
    
    
    
    is_final_tx = row['is_final_tx']
    file_path = f"/Volumes/FourTBLaCie/Yoga_Study_RADAR_4Ch_FUFD/{participant}/{radar_capture}_tx{tx_number}_{is_final_tx}.npy"
    save_file_path = file_path.replace('Yoga_Study_RADAR_4Ch_FUFD','Yoga_Study_RADAR_1CH_GOUP_BLNC_DOWN')
    save_dir = os.path.dirname(save_file_path)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    four_channel_radar_data = np.load(file_path)
    print(four_channel_radar_data.shape)
    
    frame_STABLE =  int(row['frame_stable'] - row['Start_Frame'])
    if not is_final_tx:
        frame_BREAK = int(row['frame_break'] - row['Start_Frame'])
    frame_DOWN = int(row['End_Frame'] - row['Start_Frame'])
    
    print(frame_STABLE, frame_BREAK, frame_DOWN)
    
    for i, channel in enumerate(four_channel_radar_data):
        # Slice the data
        GOUP_radar_data = channel[:frame_STABLE,:,:]
        print(GOUP_radar_data.shape)
        if not is_final_tx:
            DOWN_radar_data = channel[frame_BREAK:,:,:]
            print(DOWN_radar_data.shape)
            # Skip non balance capture
            if radar_capture == '12_MNTRR_RR_V1' and tx_number == 1:
                continue
            BLNC_radar_data = channel[frame_STABLE:frame_BREAK,:,:]
            print(BLNC_radar_data.shape)
            
        else:
            BLNC_radar_data = channel[frame_STABLE:,:,:]
            print(BLNC_radar_data.shape)
            
        # Save the different slices
        GOUP_file_path = save_file_path.replace('.npy', f'_channel{i+1}_GOUP.npy')
        np.save(GOUP_file_path, GOUP_radar_data)
        BLNC_file_path = save_file_path.replace('.npy', f'_channel{i+1}_BLNC.npy')
        np.save(BLNC_file_path, BLNC_radar_data)
        if not is_final_tx:
            DOWN_file_path = save_file_path.replace('.npy', f'_channel{i+1}_DOWN.npy')
            np.save(DOWN_file_path, DOWN_radar_data)


01_MNTRL_RR_V1
01_MNTRL_RR_V1
01_MNTRL_RR_V1
01_MNTRL_RR_V2
01_MNTRL_RR_V2
01_MNTRL_RR_V2
01_MNTRL_RR_V3
01_MNTRL_RR_V3
01_MNTRL_RR_V3
01_MNTRR_RR_V1
01_MNTRR_RR_V1
01_MNTRR_RR_V1
01_MNTRR_RR_V2
01_MNTRR_RR_V2
01_MNTRR_RR_V2
01_MNTRR_RR_V3
01_MNTRR_RR_V3
01_MNTRR_RR_V3
02_MNTRL_RR_V1
02_MNTRL_RR_V1
02_MNTRL_RR_V1
02_MNTRL_RR_V2
02_MNTRL_RR_V2
02_MNTRL_RR_V2
02_MNTRL_RR_V3
02_MNTRL_RR_V3
02_MNTRL_RR_V3
02_MNTRR_RR_V1
02_MNTRR_RR_V1
02_MNTRR_RR_V1
02_MNTRR_RR_V2
02_MNTRR_RR_V2
02_MNTRR_RR_V2
02_MNTRR_RR_V3
02_MNTRR_RR_V3
02_MNTRR_RR_V3
03_MNTRL_RR_V1
03_MNTRL_RR_V1
03_MNTRL_RR_V1
03_MNTRL_RR_V2
03_MNTRL_RR_V2
03_MNTRL_RR_V2
03_MNTRL_RR_V3
03_MNTRL_RR_V3
03_MNTRL_RR_V3
03_MNTRR_RR_V1
03_MNTRR_RR_V1
03_MNTRR_RR_V1
03_MNTRR_RR_V2
03_MNTRR_RR_V2
03_MNTRR_RR_V2
03_MNTRR_RR_V3
03_MNTRR_RR_V3
03_MNTRR_RR_V3
04_MNTRL_RR_V1
04_MNTRL_RR_V1
04_MNTRL_RR_V1
04_MNTRL_RR_V2
04_MNTRL_RR_V2
04_MNTRL_RR_V2
04_MNTRL_RR_V3
04_MNTRL_RR_V3
04_MNTRL_RR_V3
04_MNTRR_RR_V1
04_MNTRR_RR_V1
04_MNTRR_RR_V1
04_MNTRR_R